In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from os import listdir
from os.path import isfile, join

def escala_de_grises(im):
    '''
    get from: https://wizardprogrammer.blogspot.com/2019/03/convertir-imagen-blanco-y-negro-python.html
    '''
    #dimension imagen
    x = im.size[0]
    y = im.size[1]
    #Crear nueva imagen con las dimensiones de la imagen anterior
    im2 = Image.new('RGB', (x, y))
    i = 0
    while i < x:
        j = 0
        while j < y:
            #Obtenemos el valor RGB de cada pixel
            r, g, b = im.getpixel((i,j))
            #Obtenemos su equivalente en la escala de gris
            p = (r * 0.3 + g * 0.59 + b * 0.11)
            #Ese valor lo convertimos a entero
            gris = int(p)
            pixel = tuple([gris, gris, gris])
            #En la nueva imagen en la posición i, j agregamos el nuevo color 
            im2.putpixel((i,j), pixel)
            j += 1
        i += 1
    return im2

def add_blank_space_fixed(img_B_W ,random_blank, h_blank = 50 , w_blank = 50 , h_original =256, w_original=256, pos_x_blank = 0, pos_y_blank = 0):  
    '''
    Agregar espacio en blanco a la imagen input blanco y negro.
    img_B_W: imagen a extraer espacio en blanco
    random_blank: True o False, si se quiere un recorte random o no
    h_blank: tamaño horizonal del recorte
    w_blank: tamaño vertical del recorte
    h_original: tamaño de la imagen
    w_original: tamaño de la imagen
    pos_x_blank: opcional si se elegio un recorte no aleatorio, dar coordenada x al espacio en blanco
    pos_y_blank: opcional si se elegio un recorte no aleatorio, dar coordenada x al espacio en blanco
    ''' 
    img_white = Image.new('RGB', (h_blank, w_blank), "white")   
    if random_blank == True:  #recorte en blanco random
        pos_h_random = np.random.randint(h_original - h_blank)  
        pos_w_random = np.random.randint(w_original - w_blank)    
    if random_blank == False:  #coordenadas del recorte
        pos_h_random = pos_x_blank
        pos_w_random = pos_y_blank
    img_B_W.paste(img_white,(pos_h_random, pos_w_random))
    return img_B_W

def preprocess_img(final_height,final_width, blank, random_blank, path, blank_x = 100, blank_y = 100):
    '''
    Funcion que lee una imagen y redimensiona al valor deseado y luego crea una imagen blanco y negro y     la une con el original. - Crear dataset en formato deseado real-b&w
    '''
    img_read = Image.open(path)  #leer imagen
    img_resized = img_read.resize((final_height,final_width))   #reescalar imagen
    img_bw = escala_de_grises(img_resized)  #imagen reescalada en blanco y negro
    if blank == True:
        img_bw = add_blank_space_fixed(img_bw, random_blank, 77, 77, 256, 256, blank_x, blank_y) 

    #unir fotos real (izquierda) y blanco y negro (derecha)
    list_images = [img_resized, img_bw]

    final_img = Image.new('RGB', (final_width * 2, final_height))

    x_offset = 0
    for im in list_images:
        final_img.paste(im, (x_offset,0))
        x_offset += im.size[0]

    return final_img

def create_file_path(path):
    path_frame_original = [arch for arch in listdir(path) if isfile(join(path, arch))] 
    path_ok = [str(x) + '.jpg' for x in range(len(path_frame_original))]
    return path_ok


In [2]:
print('Leer frames originales, transformarlos a formato de entrada a la pix2pix')

Leer frames originales, transformarlos a formato de entrada a la pix2pix


In [3]:
#parametros
path_data = 'E:/Desktop/MLBI - 2020/Proyecto GANs/Data animals/Frames_original/dog select.mp4/'#carpeta de origen
destiny_folder = 'E:/Desktop/MLBI - 2020/Proyecto GANs/Data animals/To pix2pix/dog select.mp4/'#carpeta de destino
height = 256   #Dimensiones de las imagenes reales y de entrada
width = 256   

print('Revisión existencia de carpetas de origen y de destino\n')
if os.path.isdir(path_data):
    print("Origen. Sí es una carpeta")
else:
    print("Origen. No es una carpeta o no existe")

if os.path.isdir(destiny_folder):
    print("Destino. Sí es una carpeta")
else:
    print("Destino. No es una carpeta o no existe")

Revisión existencia de carpetas de origen y de destino

Origen. Sí es una carpeta
Destino. Sí es una carpeta


In [4]:
#crear lista con los archivos 
img_path_list = create_file_path(path_data)  

#crear imagenes
for i in range(len(img_path_list)):
    skip = int(len(img_path_list)/10)
    if i%skip == 0:
        print('Procesando imagen: ', i)
    
    try:
        imagen_to_save = preprocess_img(final_height = height, 
                                        final_width = width, 
                                        blank = True, 
                                        random_blank = False,
                                        path = path_data + img_path_list[i],     #parth origen de los frames
                                        blank_x = 100, 
                                        blank_y = 100 )
        
        imagen_to_save.save(destiny_folder + str(i)+'.jpg')   #nombre de archivo a ser guardado

    except:
        print('error with image', i)
        pass

Procesando imagen:  0
Procesando imagen:  19
Procesando imagen:  38
Procesando imagen:  57
Procesando imagen:  76
Procesando imagen:  95
Procesando imagen:  114
Procesando imagen:  133
Procesando imagen:  152
Procesando imagen:  171
Procesando imagen:  190
